In [1]:
%load_ext autoreload
%autoreload 2

In [119]:
import instructor
import google.generativeai as genai
from pydantic import BaseModel
from rich import print

client = instructor.from_gemini(genai.GenerativeModel("gemini-2.0-flash-exp"))


class StoryOutline(BaseModel):
    title: str
    description:str
    melody:str
    banner_image:str

user_prompt = "write me a story about a man who finds himself in a post apocalyptic world"


resp = client.chat.completions.create(
    response_model=StoryOutline,
    messages=[
        {
            "role":"system",
            "content": """
            You're going to be given a prompt by a user. Based off the prompt generate the following

            1. A title for a story based off the prompt
            2. A 2-3 sentence description for an initial starting point where we can start the story
            3. A 2-3 sentence description of a melody/beat that would be suitable to be played while the story is being told - this can be anything as long as it matches the story
            4. A long and descriptive description of a banner image in pixel art style that would be suitable for the story as cover art

            Remember that we are just generating a starting point for the story. We'll be generating more situations as the user progresses through the story so don't worry about the story being complete.

            Here are some examples of what the banner image description should look like
            <banner examples>
                Retro Pixel, A pixelated image of a german shepherd dog. The dogs fur is a vibrant shade of brown, with a black stripe running down its back. The background is a light green, and the dogs shadow is cast on the ground.

                Retro Pixel, A pixelated image of a man surfing on a surfboard. The mans body is covered in a red shirt and blue shorts. His arms are out to the sides of his body. The surfboard is a vibrant blue color. The water is a light blue color with white splashes. The sun is shining on the right side of the image.

                Retro Pixel, pixel art of a Hamburger in the style of an old video game, hero, pixelated 8bit, final boss 
            </banner examples>
            """
        },
        {
            "role":"user",
            "content": user_prompt
        }
    ]
)

print(resp)

StoryOutline(
    title='The Last Echo',
    description="The man wakes up in a world he doesn't recognize, buildings are destroyed and nature has begun to 
reclaim what was once a thriving city. He has no memory of how he got here, only a burning desire to find out what 
happened. His journey begins as he navigates through the ruins.",
    melody='A slow, melancholic melody with a hint of hope, played on a lone piano. The music is sparse, with long 
sustained notes that create a sense of emptiness and isolation. As the man ventures forward, the tempo increases 
and more instruments join in, adding a layer of determination and drive.',
    banner_image='Retro Pixel, A pixelated image of a lone man standing in a desolate, ruined city. The man is 
wearing tattered clothes and has a determined look on his face. The city buildings are crumbling, with broken 
windows and overgrown vegetation. The sky is a hazy, orange color, suggesting a polluted atmosphere.'
)

In [120]:
from pydantic import Field, field_validator

class Choice(BaseModel):
    title:str
    description:str

class Situation(BaseModel):
    is_terminal:bool = Field(description="Whether this is the end of the story")
    title:str = Field(description="A short title that describes the situation")
    description:str 
    image:str
    user_choices:list[Choice]

    @field_validator("user_choices")
    def validate_user_choices(cls, v):
        if len(v) < 3 or len(v) > 4:
            raise ValueError("User choices must be between 3 and 4")
        return v

class UserChoice(BaseModel):
    title: str
    description: str


def get_next_situation(story_description: str, story_title: str, previous_choices: list[UserChoice], max_depth:int=3) -> Situation:
    client = instructor.from_gemini(genai.GenerativeModel("gemini-1.5-flash-latest"))
    choices = client.chat.completions.create(
        response_model=Situation,
        messages=[
            {
                "role":"system",
                "content": """
                You're going to be given a story description and choices that a user has made up to this point. 
                
                {% if steps_remaining == 0 %}
                You must end the story in this choice. The story should reach a satisfying conclusion with no further choices available.

                Your job is to generate the following
                - A title to conclude the story thus far
                - A 2-3 sentence description for what happens now that the storyis complete
                - An image which depics the final conclusion of the story
                - An empty list for user_choices since the story is complete

                Rules
                - The story must end in this situation with a satisfying conclusion. Make sure that the user either dies, wins or reaches the end of the story by then. There cannot be any more choices available to the user.

                Here are some examples of what the image description should look like. Remember that it should showcase the final scene now that the story is complete.
                {% else %}
                Continue the story with interesting choices that progress toward a conclusion.

                Your job is to generate the following
                
                1. A title for the situation thus far
                2. A 2-3 sentence description for what happens between the previous situations and when the user is faced with the choices
                3. A list of 3-4 choices that the user can make.
                4. An image which depicts the current situation the user is facing

                Rules:
                - If you determine that the story is complete, return an empty list for the user_choices and return is_terminal as True
                - Story choices should be unique and interesting and move the story in very distinct directions
                - Write complete sentences and use proper grammar and punctuation. Make sure to output valid strings.
                - Choices descriptions should be 1-2 sentences and describe the significance of the choice
                - Choices should not bring the user back to the a previous situation
                - The story should end in {{ steps_remaining }} more choices at most. This means that the main character should either die, win, or reach the end of the story by then. There should be no more choices provided to the user ( and the user should not be able to make any more choices)

                Here are some examples of what the image description should look like. Remember that it should showcase what the current situation is like and what the user is facing from the last choice he has made when he is about to make the choice
                {% endif %}
                
                <banner examples>
                    Retro Pixel, A pixelated image of a german shepherd dog. The dogs fur is a vibrant shade of brown, with a black stripe running down its back. The background is a light green, and the dogs shadow is cast on the ground.

                    Retro Pixel, A pixelated image of a man surfing on a surfboard. The mans body is covered in a red shirt and blue shorts. His arms are out to the sides of his body. The surfboard is a vibrant blue color. The water is a light blue color with white splashes. The sun is shining on the right side of the image.

                    Retro Pixel, pixel art of a Hamburger in the style of an old video game, hero, pixelated 8bit, final boss 
                </banner examples>
                """
            },
            {
                "role":"user",
                "content": """
                Here is the initial story description that we started with

                Story title: {{ story_title }}
                Story description: {{ story_description }}

                Here are the choices that the user has made:
                {% for choice in previous_choices %}
                - {{ choice.title }} : {{ choice.description }}
                {% endfor %}

                {% if steps_remaining == 0 %}
                Take into account the choices that the user has made so far and generate a final situation that concludes the story.
                {% else %}
                
                Take into account the choices that the user has made so far and generate a new situation that progresses the story.
                {% endif %}
                """
            }
        ],
        context={
            "story_title": story_title,
            "story_description": story_description,
            "previous_choices": previous_choices,
            "steps_remaining": max_depth - len(previous_choices)
        }
    )
    
    return choices

In [121]:
choices = []
max_depth = 0
choice_1 = get_next_situation(
    "Born without the ability to be affected by magic, a young man seeks to understand the origin of his unique immunity. He spends his formative years traveling the land, seeking answers from hermits, scholars, and mages",
    "The Magicless Wanderer",
    [],
    max_depth,
)
print(choice_1)

Situation(
    is_terminal=True,
    title='The Scholar',
    description='The Wanderer, having journeyed far and wide, finally discovers an ancient text detailing 
individuals born without magical susceptibility. He learns his condition is not a curse, but a unique biological 
anomaly, a key to understanding the very fabric of magic itself. He chooses to dedicate his life to researching 
this anomaly, becoming a renowned scholar and unlocking new avenues of magical understanding.',
    image='Retro Pixel, A pixelated image of a man in a robe studying an ancient text. The man has a thoughtful 
expression on his face. The background is a dimly lit library filled with bookshelves.',
    user_choices=[
        Choice(title='', description=''),
        Choice(title='', description=''),
        Choice(title='', description='')
    ]
)

In [108]:
chosen_choice = choice_1.user_choices[0]
choices.append(
    UserChoice(
        title=chosen_choice.title,
        description=f"""
        {choice_1.description}. User chose to {chosen_choice.description}
        """.strip()
    )
)
print(choices)

[
    UserChoice(
        title='Press Onward',
        description='The man pushes through the debris, his boots crunching on broken glass and twisted metal.  He 
sees a flickering light in the distance, a beacon of hope in this desolate landscape. Will he risk venturing 
towards the unknown?. User chose to Press forward toward the light, hoping it signals safety or help. This path is 
fraught with unknown dangers, but the hope of rescue keeps him moving.'
    )
]

In [109]:
choice_2 = get_next_situation(resp.description, resp.title, choices, max_depth)
print(choice_2)

Situation(
    is_terminal=False,
    title='Wasteland Wanderer: The Abandoned Building',
    description='The flickering light leads him to an old, abandoned building.  The entrance is partially blocked 
by rubble, but a narrow passage is visible. Inside, he hears faint whispers and the metallic clang of something 
being worked on. Does he dare enter, or seek another path?',
    image='Retro Pixel, A pixelated image of an abandoned building with a flickering light in a window. Rubble 
blocks the entrance, but a narrow passage is visible. The background is dark and desolate.',
    user_choices=[
        Choice(
            title='Enter',
            description='Enter the building - The whispers and metallic sounds suggest activity inside, possibly 
help or danger.  Choosing this path could lead to unexpected encounters.'
        ),
        Choice(
            title='Seek another path',
            description='Find another path -  He could try searching for another route, avoiding the building.  
This might lead him to a different, potentially safer location, but he might miss an opportunity for help.'
        ),
        Choice(
            title='Try to communicate',
            description='Try to communicate - The man shouts into the passage to see if someone is in there. This 
is a low risk but potentially high reward action. He may find help, but he might also attract unwanted attention.'
        )
    ]
)

In [110]:
chosen_choice = choice_2.user_choices[0]
choices.append(
    UserChoice(
        title=chosen_choice.title,
        description=f"""
        {choice_2.description}. User chose to {chosen_choice.description}
        """.strip()
    )
)
print(choices)

[
    UserChoice(
        title='Press Onward',
        description='The man pushes through the debris, his boots crunching on broken glass and twisted metal.  He 
sees a flickering light in the distance, a beacon of hope in this desolate landscape. Will he risk venturing 
towards the unknown?. User chose to Press forward toward the light, hoping it signals safety or help. This path is 
fraught with unknown dangers, but the hope of rescue keeps him moving.'
    ),
    UserChoice(
        title='Enter',
        description='The flickering light leads him to an old, abandoned building.  The entrance is partially 
blocked by rubble, but a narrow passage is visible. Inside, he hears faint whispers and the metallic clang of 
something being worked on. Does he dare enter, or seek another path?. User chose to Enter the building - The 
whispers and metallic sounds suggest activity inside, possibly help or danger.  Choosing this path could lead to 
unexpected encounters.'
    )
]

In [115]:
print("We chose", choices[-1].title)
choice_3 = get_next_situation(resp.description, resp.title, choices, max_depth)
print(choice_3)


We chose Communicate

Situation(
    is_terminal=True,
    title='Wasteland Reborn',
    description='The figures, surprisingly, are not hostile. They are survivors, scavenging and rebuilding. They 
welcome the man, offering him food and shelter. Over time, he becomes an integral part of their community, helping 
to rebuild and create a new life in the wasteland. Years later, the once desolate city shows signs of life, a 
testament to human resilience and the power of community.',
    image='Retro Pixel, A pixelated image of a group of survivors working together to rebuild a city in a 
wasteland. The city is made of salvaged metal and other materials. The survivors are smiling and working 
together.',
    user_choices=[
        Choice(title='', description=''),
        Choice(title='', description=''),
        Choice(title='', description='')
    ]
)